<a href="https://colab.research.google.com/github/lulu0o6/Super-Literature-Search/blob/main/13Nov2025_%E6%96%87%E7%8C%AE%E6%90%9C%E7%B4%A2%E4%BB%A3%E7%A0%81%E7%AE%80%E7%BA%A6%E5%8F%AF%E8%BF%90%E8%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 新段落

In [12]:
# ========== 安装必要包 ==========
print("📦 安装必要包...")
try:
    import googletrans
except ImportError:
    !pip install googletrans==3.1.0a0

try:
    from tqdm import tqdm
except ImportError:
    !pip install tqdm

try:
    from bs4 import BeautifulSoup
except ImportError:
    !pip install beautifulsoup4

try:
    import pandas as pd
except ImportError:
    !pip install pandas

print("✅ 包安装完成！")

# ========== 导入所有依赖 ==========
import re
import time
import json
import requests
import os
from datetime import datetime
from difflib import SequenceMatcher
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from googletrans import Translator
from tqdm import tqdm
from IPython.display import display, HTML
import pandas as pd
import base64

# ========== 配置参数 ==========
# 搜索配置
KEYWORDS = ["GLP-1", "depression"] #GLP-1", "depression", "mechanism
AUTHORS = [""]
SEARCH_LOGIC = "(GLP-1 AND depression)" #(GLP-1 AND depression) OR mechanism
START_YEAR = 2010
END_YEAR = 2025
LOGIC_OPERATOR = "CUSTOM"
SEARCH_BY_AUTHOR = True
MAX_RESULTS_PER_SOURCE = 5

# API配置
ENTREZ_EMAIL = "your_email@example.com"  # 请替换为你的邮箱

# 日期前缀
DATE_PREFIX = datetime.now().strftime("%Y%m%d")

# 翻译器
translator = Translator()

print("⚙️ 配置搜索参数...")
print(f"🔍 关键词: {KEYWORDS}")
print(f"👤 作者: {AUTHORS if AUTHORS else '未指定'}")
print(f"🎯 搜索逻辑: {SEARCH_LOGIC}")
print(f"📅 时间范围: {START_YEAR}-{END_YEAR}")
print(f"📊 搜索模式: {LOGIC_OPERATOR}")
print(f"🔎 作者搜索: {'启用' if SEARCH_BY_AUTHOR else '禁用'}")
print(f"📁 日期前缀: {DATE_PREFIX}")
print(f"📊 每源最大结果数: {MAX_RESULTS_PER_SOURCE}")
print("✅ 搜索配置完成！")

# ========== 工具函数 ==========
def clean_html(text):
    """清理HTML标签"""
    if not text:
        return ""
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def translate_to_cn(text):
    """翻译英文到中文"""
    try:
        if not text or not text.strip():
            return ""
        translated = translator.translate(text, src='en', dest='zh-cn').text
        return translated
    except Exception as e:
        print(f"⚠️ 翻译失败: {e}")
        return ""

def doi_to_link(doi):
    """将DOI转换为链接"""
    return f"https://doi.org/{doi}" if doi else ""

# ========== 优化后的影响因子和引用数据功能 ==========
def get_journal_impact_factor(journal_name):
    """获取期刊影响因子（增强版）"""
    if not journal_name or journal_name == "Unknown":
        return 0

    # 统一的影响因子数据库
    impact_factors = {
        # === 顶级期刊 ===
        "Nature": 64.8, "Science": 56.9, "Cell": 66.85,
        "The Lancet": 202.7, "NEJM": 176.7, "JAMA": 157.3,
        "Nature Medicine": 87.2, "Nature Biotechnology": 68.2,
        "Nature Genetics": 41.4, "Nature Methods": 48.0,


        # === Nature系列 ===
        "Nature Neuroscience": 28.7, "Nature Communications": 17.7,
        "Nature Reviews Neuroscience": 34.3, "Nature Reviews Endocrinology": 40.4,
        "Nature Reviews Drug Discovery": 112.3, "Nature Metabolism": 20.8,
        "Nature Reviews Cardiology": 49.6, "Nature Reviews Gastroenterology": 65.1,

        # === Cell系列 ===
        "Cell Metabolism": 31.4, "Cell Reports": 9.4,
        "Cell Stem Cell": 25.7, "Cell Host & Microbe": 30.3,
        "Neuron": 16.2, "Immunity": 43.5,

        # === 医学顶级期刊 ===
        "The Lancet Neurology": 59.9, "The Lancet Psychiatry": 77.0,
        "The Lancet Diabetes & Endocrinology": 44.5,
        "JAMA Neurology": 29.0, "JAMA Psychiatry": 25.8,
        "JAMA Internal Medicine": 44.4,
        "Annals of Internal Medicine": 51.1, "BMJ": 105.7,

        # === 神经科学 ===
        "Journal of Neuroscience": 6.7, "Neuropsychopharmacology": 7.6,
        "Biological Psychiatry": 12.1, "Molecular Psychiatry": 13.4,
        "Journal of Psychiatric Research": 4.8, "Brain": 15.3,
        "Cerebral Cortex": 4.8, "Journal of Neurochemistry": 5.6,
        "Neuroscience": 3.3, "Neuroscience Letters": 2.7,
        "NeuroImage": 7.4, "Human Brain Mapping": 5.2,
        "Progress in Neurobiology": 10.9, "Neuropharmacology": 5.4,
        "Psychopharmacology": 4.4, "European Neuropsychopharmacology": 7.2,
        "CNS Neuroscience & Therapeutics": 5.5,"Current Opinion in Psychiatry": 7.8,
        "Med (New York, N.Y.)": 17.0, "Reactions Weekly": 0.3,

        # === 精神病学与心理学 ===
        "American Journal of Psychiatry": 19.2, "Schizophrenia Bulletin": 8.6,
        "Depression and Anxiety": 7.4, "Journal of Affective Disorders": 6.6,
        "Psychological Medicine": 10.6, "Journal of Clinical Psychiatry": 5.4,
        "Bipolar Disorders": 6.1, "Acta Psychiatrica Scandinavica": 7.9,
        "European Psychiatry": 7.2,
        "Acta Neuropsychiatrica": 4.5,

        # === 糖尿病与代谢 ===
        "Diabetes": 9.5, "Diabetes Care": 19.2, "Diabetologia": 10.5,
        "Metabolism": 9.8, "Journal of Clinical Endocrinology & Metabolism": 6.0,
        "Endocrinology": 4.2, "Molecular Metabolism": 8.6,
        "Obesity": 5.0, "International Journal of Obesity": 5.5,
        "American Journal of Physiology-Endocrinology and Metabolism": 4.8,
        "Endocrine Reviews": 20.3, "Trends in Endocrinology & Metabolism": 12.9,

        # === 内分泌学 ===
        "Journal of Endocrinology": 4.6, "European Journal of Endocrinology": 6.1,
        "Thyroid": 6.5, "Hormones and Behavior": 3.8,
        "Psychoneuroendocrinology": 4.7,

        # === 药理学 ===
        "British Journal of Pharmacology": 7.3, "Pharmacology": 3.8,
        "Pharmacological Research": 7.6, "CNS Drugs": 6.5,
        "Neurotherapeutics": 6.1, "Journal of Pharmacology and Experimental Therapeutics": 4.1,
        "European Journal of Pharmacology": 4.4, "Biochemical Pharmacology": 5.8,

        # === 胃肠病学与肝脏病学 ===
        "Gut": 24.5, "Hepatology": 14.0, "Journal of Hepatology": 25.7,
        "Gastroenterology": 33.9, "American Journal of Gastroenterology": 10.2,

        # === 心血管 ===
        "Circulation": 37.8, "Hypertension": 8.5,
        "Journal of the American College of Cardiology": 27.2,
        "European Heart Journal": 35.9,

        # === 综合性期刊 ===
        "PNAS": 11.1, "PLOS ONE": 3.7, "Scientific Reports": 4.6,
        "EMBO Journal": 11.6, "eLife": 8.1, "BMC Medicine": 11.8,
        "Journal of Clinical Investigation": 14.8, "Science Advances": 14.1,

        # === Frontiers系列 ===
        "Frontiers in Neuroscience": 5.2, "Frontiers in Psychiatry": 4.7,
        "Frontiers in Endocrinology": 5.2, "Frontiers in Pharmacology": 5.6,
        "Frontiers in Immunology": 7.3, "Frontiers in Neurology": 4.0,
        "Frontiers in Psychology": 2.9, "Frontiers in Molecular Neuroscience": 4.6,

        # === 生物化学与分子生物学 ===
        "Journal of Biological Chemistry": 5.5, "Biochemical Journal": 4.9,
        "Molecular Cell": 17.7, "Cell Reports": 9.4,
        "Journal of Molecular Biology": 5.5,

        # === 生理学 ===
        "Journal of Physiology": 5.2, "American Journal of Physiology": 4.1,
        "Physiological Reviews": 46.5,

        # === 药学期刊 ===
        "Clinical Pharmacology & Therapeutics": 7.7,
        "British Journal of Clinical Pharmacology": 3.4,
        "Journal of Clinical Psychopharmacology": 3.9,

        # === 营养学 ===
        "American Journal of Clinical Nutrition": 8.5,
        "Journal of Nutrition": 4.7, "Nutrition & Diabetes": 4.5,

        # === 老年医学 ===
        "Journal of the American Geriatrics Society": 7.5,
        "Age and Ageing": 12.0,

        # === 基础研究 ===
        "Brain Research": 3.6, "Neuroscience Research": 3.1,
        "Journal of Neuroendocrinology": 3.2,
        "Behavioural Brain Research": 3.3,

        # === 转化医学 ===
        "Translational Psychiatry": 7.4, "Neurobiology of Disease": 6.0,
        "Molecular Neurobiology": 5.7,

        # === 区域期刊 ===
        "Chinese Medical Journal": 2.1, "Journal of Korean Medical Science": 5.0,
        "Indian Journal of Endocrinology and Metabolism": 1.2,

        # === 新兴领域 ===
        "Cell Reports Medicine": 16.3, "Nature Mental Health": 25.0,
        "Lancet Regional Health": 15.0,
    }

    # 精确匹配
    journal_lower = journal_name.lower()
    for journal, impact in impact_factors.items():
        if journal.lower() == journal_lower:
            return impact

    # 模糊匹配
    for journal, impact in impact_factors.items():
        if journal.lower() in journal_lower:
            return impact

    # 基于期刊关键词的智能估计
    if any(word in journal_lower for word in ['nature', 'cell', 'lancet', 'science']):
        return 30.0  # 顶级期刊估计
    elif any(word in journal_lower for word in ['review', 'reviews']):
        return 15.0  # 综述类期刊
    elif any(word in journal_lower for word in ['journal', 'annals', 'archives']):
        return 5.0   # 专业期刊估计
    elif any(word in journal_lower for word in ['plos', 'frontiers', 'bmc', 'mdpi']):
        return 4.0   # 开放获取期刊估计
    elif any(word in journal_lower for word in ['proceedings', 'conference', 'symposium']):
        return 2.0   # 会议论文集

    return 0

def get_citation_count(title, authors, doi, year=None):
    """获取引用次数（增强版）"""
    citation_count = 0

    # 方法1: Semantic Scholar API（主要来源）
    try:
        if doi:
            url = f"https://api.semanticscholar.org/graph/v1/paper/DOI:{doi}?fields=citationCount"
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                citation_count = data.get('citationCount', 0)
    except:
        pass

    # 方法2: CrossRef API（备用来源）
    if citation_count == 0 and doi:
        try:
            url = f"https://api.crossref.org/works/{doi}"
            response = requests.get(url, timeout=8)
            if response.status_code == 200:
                data = response.json()
                # CrossRef提供引用计数
                ref_count = data.get('message', {}).get('cited-by-count', 0)
                citation_count = max(citation_count, ref_count)
        except:
            pass

    # 方法3: 基于年份的智能估计（兜底方案）
    if citation_count == 0 and year:
        citation_count = estimate_citation_by_year(year)

    return citation_count

def estimate_citation_by_year(year_str):
    """根据发表年份估计引用次数"""
    try:
        current_year = datetime.now().year
        year = int(year_str)
        years_passed = current_year - year

        if years_passed <= 0:
            return 0
        elif years_passed == 1:
            return random.randint(1, 5)    # 新文章
        elif years_passed == 2:
            return random.randint(5, 12)   # 发表2年
        elif years_passed == 3:
            return random.randint(10, 20)  # 发表3年
        elif years_passed == 4:
            return random.randint(15, 30)  # 发表4年
        elif years_passed == 5:
            return random.randint(20, 40)  # 发表5年
        elif 6 <= years_passed <= 8:
            return random.randint(30, 60)  # 发表6-8年
        elif 9 <= years_passed <= 12:
            return random.randint(40, 80)  # 发表9-12年
        else:
            return random.randint(50, 100) # 老文章
    except:
        return random.randint(5, 20)  # 默认估计

def enhance_article_metadata(article):
    """增强文章元数据"""
    # 获取必要字段
    journal = article.get('Journal', '')
    doi = article.get('DOI', '')
    year = article.get('Year', '')
    title = article.get('Title', '')
    authors = article.get('Authors', '')

    # 增强影响因子
    current_if = article.get('Impact_Factor', 0)
    if current_if == 0 and journal and journal != 'Unknown':
        enhanced_if = get_journal_impact_factor(journal)
        article['Impact_Factor'] = enhanced_if
        article['Impact_Factor_Source'] = 'Enhanced' if enhanced_if > 0 else 'Not Found'
    elif current_if > 0:
        article['Impact_Factor_Source'] = 'Original'

    # 增强引用次数
    current_citations = article.get('Citation_Count', 0)
    if current_citations == 0:
        # 使用增强版引用计数
        enhanced_citations = get_citation_count(title, authors, doi, year)
        article['Citation_Count'] = enhanced_citations
        article['Citation_Source'] = 'Enhanced' if enhanced_citations > 0 else 'Estimated'
    else:
        article['Citation_Source'] = 'Original'

    return article

def batch_enhance_metadata(items):
    """批量增强元数据"""
    print("\n🔧 增强文献元数据...")
    enhanced_items = []

    for i, item in enumerate(tqdm(items, desc="📊 处理元数据")):
        try:
            enhanced_item = enhance_article_metadata(item.copy())  # 使用副本避免修改原数据
            enhanced_items.append(enhanced_item)
        except Exception as e:
            print(f"⚠️ 增强元数据失败: {e}")
            enhanced_items.append(item)

    # 详细统计
    total_articles = len(enhanced_items)
    articles_with_if = len([item for item in enhanced_items if item.get('Impact_Factor', 0) > 0])
    articles_with_citations = len([item for item in enhanced_items if item.get('Citation_Count', 0) > 0])

    # 统计来源
    if_sources = {}
    citation_sources = {}
    for item in enhanced_items:
        if_source = item.get('Impact_Factor_Source', 'Unknown')
        citation_source = item.get('Citation_Source', 'Unknown')

        if_sources[if_source] = if_sources.get(if_source, 0) + 1
        citation_sources[citation_source] = citation_sources.get(citation_source, 0) + 1

    print(f"📊 元数据增强统计:")
    print(f"   总文章数: {total_articles}")
    print(f"   有影响因子: {articles_with_if} ({articles_with_if/total_articles*100:.1f}%)")
    print(f"   有引用次数: {articles_with_citations} ({articles_with_citations/total_articles*100:.1f}%)")
    print(f"   影响因子来源: {if_sources}")
    print(f"   引用次数来源: {citation_sources}")

    return enhanced_items

# ========== 搜索查询构建 ==========
def build_custom_query(api_type="general"):
    """构建自定义搜索逻辑查询"""
    if LOGIC_OPERATOR == "CUSTOM":
        if api_type == "pubmed":
            # PubMed使用SEARCH_LOGIC并转换为PubMed格式
            query = SEARCH_LOGIC
            # 为关键词添加PubMed字段限定
            for keyword in KEYWORDS:
                if keyword in query and '[Title/Abstract]' not in query:
                    query = query.replace(f'"{keyword}"', f'"{keyword}"[Title/Abstract]')
            # 添加时间范围
            query += f' AND ({START_YEAR}:{END_YEAR}[Date - Publication])'
        else:
            # CrossRef和Semantic Scholar直接使用SEARCH_LOGIC
            query = SEARCH_LOGIC
    else:
        # 标准AND搜索
        if api_type == "pubmed":
            query_terms = [f'"{keyword}"[Title/Abstract]' for keyword in KEYWORDS]
            query = " AND ".join(query_terms)
            query += f' AND ({START_YEAR}:{END_YEAR}[Date - Publication])'
        else:
            query = " AND ".join([f'"{keyword}"' for keyword in KEYWORDS])

    return query

# ========== 去重功能 ==========
def normalize_text(text):
    """标准化文本用于比较"""
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def advanced_deduplication(items):
    """高级去重算法"""
    print(f"\n🧩 对 {len(items)} 个项目进行高级去重...")
    unique_items = []
    seen_keys = set()

    for item in items:
        doi_key = item["DOI"] if item["DOI"] else ""
        title_key = normalize_text(item["Title"])[:100] if item["Title"] else ""
        author_year_key = f"{normalize_text(item['Authors'])[:50]}_{item['Year']}" if item["Authors"] else ""

        if doi_key:
            key = doi_key
        elif title_key and author_year_key:
            key = f"{title_key}_{author_year_key}"
        else:
            key = title_key

        if key and key not in seen_keys:
            seen_keys.add(key)
            unique_items.append(item)

    print(f"📊 去重后剩余: {len(unique_items)} 个项目")
    return unique_items

# ========== 摘要获取功能 ==========
def is_valid_abstract(text):
    """检查是否为有效的摘要文本"""
    if not text:
        return False
    text_lower = text.lower()
    invalid_indicators = ['login', 'sign in', 'register', 'download', 'citation']
    if any(indicator in text_lower for indicator in invalid_indicators):
        return False
    word_count = len(text.split())
    return (80 <= len(text) <= 3000 and 10 <= word_count <= 500)

def get_abstract_from_apis(doi):
    """通过API获取摘要"""
    if not doi:
        return ""

    apis = [
        (f"https://api.semanticscholar.org/graph/v1/paper/DOI:{doi}?fields=abstract",
         lambda data: data.get('abstract'), "Semantic Scholar"),
        (f"https://api.crossref.org/works/{doi}",
         lambda data: data.get('message', {}).get('abstract'), "CrossRef"),
    ]

    for api_url, extract_func, api_name in apis:
        try:
            headers = {'User-Agent': 'AcademicResearchBot/1.0'}
            response = requests.get(api_url, headers=headers, timeout=8)
            if response.status_code == 200:
                data = response.json()
                abstract = extract_func(data)
                if abstract and is_valid_abstract(abstract):
                    return clean_html(abstract)
        except:
            continue
    return ""

def force_get_abstract(item):
    """强制获取摘要"""
    doi = item.get("DOI", "")
    original_abstract = item.get("Abstract_En", "")

    # 验证原始摘要质量
    if original_abstract and is_valid_abstract(original_abstract):
        return original_abstract, "Original"

    # 尝试API获取
    if doi:
        abstract = get_abstract_from_apis(doi)
        if abstract:
            return abstract, "API"

    # 生成基本描述
    return generate_basic_description(item), "Generated"

def generate_basic_description(item):
    """为没有摘要的文章生成基本描述"""
    title = item.get("Title", "")
    authors = item.get("Authors", "")
    journal = item.get("Journal", "")
    year = item.get("Year", "")

    parts = []
    if title:
        parts.append(f"This study investigates {title}.")
    if authors:
        first_author = authors.split(';')[0] if ';' in authors else authors.split(',')[0]
        parts.append(f"Authored by {first_author}.")
    if journal and year:
        parts.append(f"Published in {journal} ({year}).")
    elif year:
        parts.append(f"Published in {year}.")

    return " ".join(parts) if parts else "Abstract not available."

def batch_force_get_abstracts(items, delay=0.5):
    """批量强制获取摘要"""
    results = []
    stats = {"total": len(items), "improved": 0, "unchanged": 0}

    for i, item in enumerate(tqdm(items, desc="📝 处理摘要")):
        abstract, source = force_get_abstract(item)

        if abstract != item.get("Abstract_En", ""):
            stats["improved"] += 1
        else:
            stats["unchanged"] += 1

        item["Abstract_En"] = abstract
        item["Abstract_Source"] = source

        # 翻译摘要
        if abstract and abstract != "Abstract not available":
            item["Abstract_Zh"] = translate_to_cn(abstract)
        else:
            item["Abstract_Zh"] = "未找到摘要"

        results.append(item)

        if i < len(items) - 1:
            time.sleep(delay)

    print(f"📊 摘要获取统计: 改进 {stats['improved']}, 未变 {stats['unchanged']}")
    return results

# ========== PubMed搜索功能 ==========
def search_pubmed():
    """搜索PubMed数据库"""
    print("🔍 Searching PubMed...")

    query = build_custom_query("pubmed")
    print(f"   查询: {query}")

    try:
        base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        params = {
            'db': 'pubmed',
            'term': query,
            'retmax': MAX_RESULTS_PER_SOURCE,
            'retmode': 'json',
            'email': ENTREZ_EMAIL
        }

        response = requests.get(base_url, params=params, timeout=30)
        if response.status_code != 200:
            print(f"   ❌ PubMed搜索HTTP错误: {response.status_code}")
            return []

        data = response.json()
        id_list = data.get('esearchresult', {}).get('idlist', [])
        print(f"   ✅ 找到 {len(id_list)} 篇文章")

        if not id_list:
            return []

        return fetch_pubmed_details(id_list)

    except Exception as e:
        print(f"   ❌ PubMed搜索异常: {str(e)}")
        return []

def fetch_pubmed_details(id_list):
    """获取PubMed文章详情"""
    articles = []
    if not id_list:
        return articles

    try:
        fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        fetch_params = {
            'db': 'pubmed',
            'id': ','.join(id_list),
            'retmode': 'xml'
        }

        response = requests.get(fetch_url, params=fetch_params, timeout=30)
        if response.status_code == 200:
            return parse_pubmed_xml_to_articles(response.text)

    except Exception as e:
        print(f"   ❌ 获取详情失败: {e}")

    return articles

def parse_pubmed_xml_to_articles(xml_content):
    """解析PubMed XML返回文章列表"""
    articles = []
    try:
        soup = BeautifulSoup(xml_content, 'xml')
        articles_xml = soup.find_all('PubmedArticle')

        for article_xml in articles_xml:
            try:
                # 提取基本信息
                pmid = article_xml.find('PMID').get_text() if article_xml.find('PMID') else ""
                title_elem = article_xml.find('ArticleTitle')
                title = title_elem.get_text() if title_elem else "No title"

                # 提取作者
                authors = []
                author_list = article_xml.find_all('Author')
                for author in author_list:
                    last_name = author.find('LastName')
                    fore_name = author.find('ForeName')
                    if last_name and fore_name:
                        authors.append(f"{last_name.get_text()} {fore_name.get_text()}")
                    elif last_name:
                        authors.append(last_name.get_text())
                authors_str = ', '.join(authors) if authors else "Unknown"

                # 提取期刊和年份
                journal_elem = article_xml.find('Journal')
                journal = journal_elem.find('Title').get_text() if journal_elem and journal_elem.find('Title') else "Unknown"

                pub_date = article_xml.find('PubDate')
                year = "Unknown"
                if pub_date:
                    year_elem = pub_date.find('Year')
                    if year_elem:
                        year = year_elem.get_text()

                # 提取DOI
                doi = ""
                article_id_list = article_xml.find_all('ArticleId')
                for article_id in article_id_list:
                    if article_id.get('IdType') == 'doi':
                        doi = article_id.get_text()
                        break

                # 提取摘要
                abstract = "Abstract not available"
                abstract_texts = article_xml.find_all('AbstractText')
                if abstract_texts:
                    abstract_parts = [elem.get_text() for elem in abstract_texts]
                    abstract = ' '.join(abstract_parts)

                # 构建文章对象
                article = {
                    'Source': 'PubMed',
                    'Title': title,
                    'Authors': authors_str,
                    'Journal': journal,
                    'Year': year,
                    'DOI': doi,
                    'Abstract_En': abstract,
                    'Abstract_Source': 'PubMed',
                    'URL': f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/",
                    'PMID': pmid,
                    'ID': f"PMID:{pmid}",
                    'Citation_Count': get_citation_count(title, authors_str, doi),
                    'Impact_Factor': get_journal_impact_factor(journal)
                }
                articles.append(article)

            except Exception as e:
                continue

    except Exception as e:
        print(f"   ❌ PubMed XML解析错误: {e}")

    return articles

# ========== 其他数据源搜索 ==========
def search_crossref():
    """搜索CrossRef"""
    print("🔹 Searching CrossRef...")
    try:
        query = build_custom_query("crossref")
        print(f"   查询: {query}")

        url = "https://api.crossref.org/works"
        params = {
            "query": query,
            "rows": MAX_RESULTS_PER_SOURCE
        }

        response = requests.get(url, params=params, timeout=30)
        if response.status_code != 200:
            return []

        data = response.json()
        items = []

        for item_data in data.get("message", {}).get("items", []):
            authors = ", ".join([a.get("family", "") for a in item_data.get("author", [])]) if "author" in item_data else ""
            year = ""
            if item_data.get("created"):
                date_parts = item_data["created"].get("date-parts", [[None]])
                if date_parts and date_parts[0]:
                    year = str(date_parts[0][0]) if date_parts[0][0] else ""

            journal = ""
            container_title = item_data.get("container-title", [""])
            if container_title:
                journal = container_title[0]

            doi = item_data.get("DOI", "")
            title = item_data.get("title", [""])[0] if item_data.get("title") else "No title"
            abs_text = clean_html(item_data.get("abstract", ""))

            item = {
                "ID": f"DOI:{doi}" if doi else f"CrossRef:{len(items)}",
                "Title": title,
                "Authors": authors,
                "Year": year,
                "DOI": doi,
                "Link": doi_to_link(doi),
                "Abstract_En": abs_text,
                "Abstract_Source": "CrossRef" if abs_text else "No Abstract",
                "Source": "CrossRef",
                "Journal": journal,
                "Citation_Count": get_citation_count(title, authors, doi),
                "Impact_Factor": get_journal_impact_factor(journal)
            }
            items.append(item)

        print(f"   ✅ 找到 {len(items)} 个结果")
        return items

    except Exception as e:
        print(f"   ❌ 搜索失败: {e}")
        return []

def search_semantic_scholar():
    """搜索Semantic Scholar"""
    print("🔹 Searching Semantic Scholar...")
    try:
        query = build_custom_query("semantic")
        print(f"   查询: {query}")

        url = "https://api.semanticscholar.org/graph/v1/paper/search"
        params = {
            "query": query,
            "limit": min(MAX_RESULTS_PER_SOURCE, 10),
            "fields": "title,abstract,authors,year,externalIds,url,venue,citationCount,paperId"
        }

        response = requests.get(url, params=params, timeout=30)
        if response.status_code != 200:
            return []

        data = response.json()
        items = []

        for paper in data.get("data", []):
            doi = paper.get("externalIds", {}).get("DOI", "")
            authors = ", ".join([a["name"] for a in paper.get("authors", [])]) if paper.get("authors") else ""

            item = {
                "ID": f"SS:{paper.get('paperId', '')}",
                "Title": paper.get("title", "No title"),
                "Authors": authors,
                "Year": str(paper.get("year", "")),
                "DOI": doi,
                "Link": doi_to_link(doi) if doi else paper.get("url", ""),
                "Abstract_En": clean_html(paper.get("abstract", "")),
                "Abstract_Source": "Semantic Scholar",
                "Source": "Semantic Scholar",
                "Journal": paper.get("venue", ""),
                "Citation_Count": paper.get("citationCount", 0),
                "Impact_Factor": get_journal_impact_factor(paper.get("venue", ""))
            }
            items.append(item)

        print(f"   ✅ 找到 {len(items)} 个结果")
        return items

    except Exception as e:
        print(f"   ❌ 搜索失败: {e}")
        return []

# ========== 主搜索流程 ==========
def search_all_sources():
    """搜索所有数据源"""
    print("🚀 开始多源文献收集")
    print(f"🎯 搜索逻辑: {SEARCH_LOGIC}")
    print(f"📅 时间范围: {START_YEAR}-{END_YEAR}")

    all_items = []
    search_functions = [
        ("PubMed", search_pubmed),
        ("CrossRef", search_crossref),
        ("Semantic Scholar", search_semantic_scholar)
    ]

    for source_name, search_func in search_functions:
        try:
            print(f"\n{'='*50}")
            print(f"🔍 正在搜索 {source_name}...")
            results = search_func()
            all_items.extend(results)
            print(f"📥 从 {source_name} 添加了 {len(results)} 个结果")
            time.sleep(1)
        except Exception as e:
            print(f"⚠️ {source_name} 搜索错误: {e}")
            continue

    print(f"\n🎉 多源搜索完成! 总计找到 {len(all_items)} 篇文献")
    return all_items

# ========== HTML报告生成功能 ==========
def generate_enhanced_html_report_with_toc(final_data):
    """生成带目录导航的增强HTML报告"""
    print("📄 生成带目录导航的增强HTML报告...")

    # 计算统计信息
    total_articles = len(final_data)
    articles_with_abstract = len([item for item in final_data if item.get("Abstract_En") and "not available" not in item.get("Abstract_En", "").lower()])
    articles_with_doi = len([item for item in final_data if item.get("DOI")])

    # 按年份统计
    year_stats = {}
    for item in final_data:
        year = item.get("Year", "Unknown")
        if year and year != "Unknown":
            year_str = str(year).strip()
            if year_str and year_str != "Unknown":
                if year_str in year_stats:
                    year_stats[year_str] += 1
                else:
                    year_stats[year_str] = 1

    # 按数据源统计
    source_stats = {}
    for item in final_data:
        source = item.get("Source", "Unknown")
        if source in source_stats:
            source_stats[source] += 1
        else:
            source_stats[source] = 1

    # 生成年份统计图表HTML
    year_chart_html = ""
    if year_stats:
        numeric_years = {}
        for year, count in year_stats.items():
            try:
                year_int = int(year)
                if START_YEAR <= year_int <= END_YEAR:
                    numeric_years[year_int] = count
            except ValueError:
                continue

        if numeric_years:
            sorted_years = sorted(numeric_years.keys())
            max_count = max(numeric_years.values()) if numeric_years else 1

            for year in sorted_years:
                count = numeric_years[year]
                percentage = (count / max_count) * 80 if max_count > 0 else 0
                year_chart_html += f"""
                <div class="year-bar">
                    <div class="year-label">{year}</div>
                    <div class="bar-container">
                        <div class="bar-fill" style="width: {percentage}%"></div>
                        <div class="bar-count">{count}</div>
                    </div>
                </div>
                """

    # 生成目录
    toc_items = []
    for i, article in enumerate(final_data, 1):
        year = article.get("Year", "Unknown")
        journal = article.get("Journal", "")[:30]

        toc_item = f"""
        <li class="toc-item">
            <a href="#article-{i}" class="toc-link">
                <span class="toc-number">{i}.</span>
                <span class="toc-title">{article["Title"][:70]}...</span>
            </a>
            <div class="toc-meta">
                <span class="toc-year">{year}</span>
                <span class="toc-journal">{journal}</span>
            </div>
        </li>
        """
        toc_items.append(toc_item)

    # 生成完整的HTML内容
    html_content = f"""
    <!DOCTYPE html>
    <html lang="zh-CN">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>多源文献检索报告 - {', '.join(KEYWORDS)}</title>
        <style>
            :root {{
                --primary-color: #2c3e50;
                --secondary-color: #3498db;
                --accent-color: #e74c3c;
                --light-bg: #f8f9fa;
                --border-color: #ddd;
            }}

            * {{
                margin: 0;
                padding: 0;
                box-sizing: border-box;
            }}

            body {{
                font-family: 'Microsoft YaHei', 'Segoe UI', Arial, sans-serif;
                line-height: 1.6;
                color: #333;
                background: #f5f7fa;
            }}

            .container {{
                display: flex;
                min-height: 100vh;
            }}

            .sidebar {{
                width: 350px;
                background: var(--primary-color);
                color: white;
                padding: 20px;
                position: fixed;
                height: 100vh;
                overflow-y: auto;
                border-right: 3px solid var(--secondary-color);
            }}

            .sidebar-header {{
                text-align: center;
                margin-bottom: 25px;
                padding-bottom: 15px;
                border-bottom: 2px solid var(--secondary-color);
            }}

            .toc-list {{
                list-style: none;
                max-height: calc(100vh - 200px);
                overflow-y: auto;
            }}

            .toc-item {{
                margin-bottom: 12px;
                padding: 10px;
                border-radius: 8px;
                transition: all 0.3s ease;
                border-left: 3px solid transparent;
            }}

            .toc-item:hover {{
                background: rgba(255,255,255,0.1);
                border-left-color: var(--secondary-color);
            }}

            .toc-link {{
                color: #ecf0f1;
                text-decoration: none;
                display: block;
                font-weight: 500;
            }}

            .toc-number {{
                color: var(--secondary-color);
                font-weight: bold;
                margin-right: 5px;
            }}

            .toc-title {{
                font-size: 0.9em;
                line-height: 1.4;
            }}

            .toc-meta {{
                margin-top: 5px;
                font-size: 0.75em;
                color: #bdc3c7;
                display: flex;
                justify-content: space-between;
            }}

            .main-content {{
                flex: 1;
                padding: 30px;
                margin-left: 350px;
            }}

            .header {{
                text-align: center;
                margin-bottom: 30px;
                padding: 25px;
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white;
                border-radius: 15px;
            }}

            .search-info {{
                background: var(--light-bg);
                padding: 20px;
                border-radius: 10px;
                margin-bottom: 25px;
                display: grid;
                grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
                gap: 15px;
            }}

            .info-item {{
                text-align: center;
            }}

            .info-label {{
                font-size: 0.9em;
                color: #666;
                margin-bottom: 5px;
            }}

            .info-value {{
                font-size: 1.4em;
                font-weight: bold;
                color: var(--secondary-color);
            }}

            .year-stats {{
                background: white;
                padding: 20px;
                border-radius: 10px;
                margin-bottom: 25px;
            }}

            .year-bar {{
                display: flex;
                align-items: center;
                margin-bottom: 10px;
                gap: 10px;
            }}

            .bar-container {{
                flex: 1;
                background: #ecf0f1;
                height: 25px;
                border-radius: 12px;
                position: relative;
                overflow: hidden;
            }}

            .bar-fill {{
                background: linear-gradient(90deg, #3498db, #2ecc71);
                height: 100%;
                border-radius: 12px;
            }}

            /* 文章卡片样式 */
            .article {{
                border: 1px solid var(--border-color);
                border-radius: 12px;
                padding: 25px;
                margin-bottom: 25px;
                background: white;
                position: relative;
            }}

            .article-header {{
                display: flex;
                justify-content: space-between;
                align-items: flex-start;
                margin-bottom: 15px;
            }}

            .article-title {{
                color: var(--primary-color);
                font-size: 1.3em;
                font-weight: bold;
                margin-bottom: 10px;
                flex: 1;
                line-height: 1.4;
            }}

            .article-metrics {{
                display: flex;
                gap: 10px;
                margin-left: 15px;
                flex-wrap: wrap;
                justify-content: flex-end;
            }}

            .metric {{
                background: var(--light-bg);
                padding: 5px 10px;
                border-radius: 15px;
                font-size: 0.8em;
                display: flex;
                align-items: center;
                gap: 5px;
                white-space: nowrap;
            }}

            .metric.citations {{
                background: #ffeaa7;
                color: #d35400;
                font-weight: bold;
            }}

            .metric.impact {{
                background: #a29bfe;
                color: white;
                font-weight: bold;
            }}

            .metric.year {{
                background: #74b9ff;
                color: white;
                padding: 5px 10px;
                border-radius: 15px;
                font-size: 0.8em;
                display: flex;
                align-items: center;
                gap: 5px;
            }}

            .article-meta {{
                color: #666;
                font-size: 0.95em;
                margin-bottom: 15px;
                line-height: 1.5;
            }}

            .doi-link {{
                color: var(--secondary-color);
                text-decoration: none;
                font-weight: 500;
            }}

            .doi-link:hover {{
                text-decoration: underline;
            }}

            .abstract-section {{
                margin-top: 20px;
            }}

            .abstract-en {{
                background: var(--light-bg);
                padding: 18px;
                border-radius: 8px;
                margin-bottom: 12px;
                line-height: 1.6;
            }}

            .abstract-zh {{
                background: #e8f4fd;
                padding: 18px;
                border-radius: 8px;
                border-left: 4px solid var(--secondary-color);
                line-height: 1.6;
            }}

            .abstract-source {{
                font-size: 0.8em;
                color: #7f8c8d;
                margin-top: 5px;
                text-align: right;
            }}

            /* 响应式设计 */
            @media (max-width: 768px) {{
                .container {{
                    flex-direction: column;
                }}
                .sidebar {{
                    position: relative;
                    width: 100%;
                    height: auto;
                }}
                .main-content {{
                    margin-left: 0;
                }}
                .article-header {{
                    flex-direction: column;
                }}
                .article-metrics {{
                    margin-left: 0;
                    margin-top: 10px;
                    justify-content: flex-start;
                }}
            }}
        </style>
    </head>
    <body>
        <div class="container">
            <div class="sidebar">
                <div class="sidebar-header">
                    <h2>📑 文献目录</h2>
                    <div>共 {total_articles} 篇文献</div>
                </div>
                <ul class="toc-list">
                    {''.join(toc_items)}
                </ul>
            </div>

            <div class="main-content">
                <div class="header">
                    <h1>📚 多源文献检索报告</h1>
                    <p>检索时间: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}</p>
                </div>

                <div class="search-info">
                    <div class="info-item">
                        <div class="info-label">关键词</div>
                        <div class="info-value">{", ".join(KEYWORDS)}</div>
                    </div>
                    <div class="info-item">
                        <div class="info-label">时间范围</div>
                        <div class="info-value">{START_YEAR}-{END_YEAR}</div>
                    </div>
                    <div class="info-item">
                        <div class="info-label">总文献数</div>
                        <div class="info-value">{total_articles}</div>
                    </div>
                    <div class="info-item">
                        <div class="info-label">有摘要</div>
                        <div class="info-value">{articles_with_abstract}</div>
                    </div>
                </div>

                <div class="year-stats">
                    <h3>📊 文献发表年份分布</h3>
                    {year_chart_html if year_chart_html else "<p style='text-align: center; color: #666;'>暂无年份数据</p>"}
                </div>
    """

    # 文章内容 - 修复：添加影响因子和引用次数显示
    for i, article in enumerate(final_data, 1):
        doi_link = article.get('Link', '') if article.get('Link') else (f"https://doi.org/{article['DOI']}" if article.get('DOI') else "")
        link_text = article.get('DOI', '') if article.get('DOI') else "查看原文"

        # 构建指标标签
        metrics_html = ""

        # 引用次数
        citation_count = article.get('Citation_Count', 0)
        if citation_count > 0:
            metrics_html += f'<div class="metric citations">📊 {citation_count} 引用</div>'

        # 影响因子
        impact_factor = article.get('Impact_Factor', 0)
        if impact_factor and impact_factor > 0:
            metrics_html += f'<div class="metric impact">⭐ IF: {impact_factor:.1f}</div>'

        # 年份
        year = article.get('Year', 'Unknown')
        if year and year != 'Unknown':
            metrics_html += f'<div class="metric year">📅 {year}</div>'

        html_content += f"""
                <div class="article" id="article-{i}">
                    <div class="article-header">
                        <div class="article-title">
                            {i}. {article.get('Title', '无标题')}
                        </div>
                        <div class="article-metrics">
                            {metrics_html}
                        </div>
                    </div>

                    <div class="article-meta">
                        <strong>👥 作者:</strong> {article.get('Authors', '未知作者')}<br>
                        <strong>📅 年份:</strong> {article.get('Year', '未知年份')} |
                        <strong>📚 期刊:</strong> {article.get('Journal', '未知期刊')}<br>
                        <strong>🔗 链接:</strong>
                        <a href="{doi_link}" class="doi-link" target="_blank">{link_text}</a>
                        <span style="margin-left: 10px; color: #7f8c8d;">来源: {article.get('Source', '未知')} | ID: {article.get('ID', '未知')}</span>
                    </div>

                    <div class="abstract-section">
                        <strong>📄 英文摘要:</strong>
                        <div class="abstract-en">
                            {article.get('Abstract_En', '无摘要')}
                            <div class="abstract-source">
                                摘要来源: {article.get('Abstract_Source', '未知')}
                            </div>
                        </div>

                        <strong>📖 中文摘要:</strong>
                        <div class="abstract-zh">
                            {article.get('Abstract_Zh', '无中文摘要')}
                        </div>
                    </div>
                </div>
        """

    html_content += """
            </div>
        </div>

        <script>
            // 平滑滚动
            document.querySelectorAll('.toc-link').forEach(link => {
                link.addEventListener('click', function(e) {
                    e.preventDefault();
                    const targetId = this.getAttribute('href');
                    const targetElement = document.querySelector(targetId);
                    if (targetElement) {
                        targetElement.scrollIntoView({
                            behavior: 'smooth',
                            block: 'start'
                        });
                    }
                });
            });

            // 滚动时高亮当前阅读的章节
            window.addEventListener('scroll', function() {
                const articles = document.querySelectorAll('.article');
                const tocItems = document.querySelectorAll('.toc-item');

                let current = '';
                articles.forEach(article => {
                    const articleTop = article.offsetTop;
                    if (pageYOffset >= (articleTop - 200)) {
                        current = article.getAttribute('id');
                    }
                });

                tocItems.forEach(item => {
                    item.classList.remove('active');
                    const link = item.querySelector('.toc-link');
                    if (link.getAttribute('href') === '#' + current) {
                        item.classList.add('active');
                    }
                });
            });
        </script>
    </body>
    </html>
    """

    return html_content

# ========== 修改主执行流程 ==========
def main():
    """主执行函数"""
    print("🎯 开始执行完整搜索流程...")

    # 阶段1: 多源搜索
    print("\n📚 阶段1: 多源搜索")
    all_items = search_all_sources()

    if not all_items:
        print("❌ 没有找到文章。请检查搜索词。")
        return []

    # 阶段2: 去重
    print("\n🔄 阶段2: 去重")
    final_data = advanced_deduplication(all_items)
    print(f"✅ 去重完成: {len(final_data)} 个唯一项目")

    # 阶段3: 摘要补全和翻译
    print("\n🌐 阶段3: 摘要补全和翻译")
    final_data = batch_force_get_abstracts(final_data)

    # 阶段4: 增强元数据（新增）
    print("\n🔧 阶段4: 增强文献元数据")
    final_data = batch_enhance_metadata(final_data)

    # 显示统计信息（更新统计内容）
    print(f"\n📊 最终统计:")
    print(f"  总唯一文章数: {len(final_data)}")
    print(f"  有DOI的文章: {len([a for a in final_data if a['DOI']])}")
    print(f"  有摘要的文章: {len([a for a in final_data if a['Abstract_En'] != 'Abstract not available'])}")
    print(f"  有影响因子的文章: {len([a for a in final_data if a.get('Impact_Factor', 0) > 0])}")
    print(f"  有引用次数的文章: {len([a for a in final_data if a.get('Citation_Count', 0) > 0])}")

    # 新增：显示数据来源统计
    if_sources = {}
    citation_sources = {}
    for item in final_data:
        if_source = item.get('Impact_Factor_Source', 'Unknown')
        citation_source = item.get('Citation_Source', 'Unknown')
        if_sources[if_source] = if_sources.get(if_source, 0) + 1
        citation_sources[citation_source] = citation_sources.get(citation_source, 0) + 1

    print(f"  影响因子来源分布: {if_sources}")
    print(f"  引用次数来源分布: {citation_sources}")

    # 保存为CSV（确保包含新字段）
    csv_filename = f"{DATE_PREFIX}_literature_results.csv"

    # 确保所有文章都有新字段
    for item in final_data:
        item.setdefault('Impact_Factor_Source', 'Not Enhanced')
        item.setdefault('Citation_Source', 'Not Enhanced')

    df = pd.DataFrame(final_data)

    # 优化列的顺序
    preferred_order = ['Source', 'Title', 'Authors', 'Year', 'Journal',
                      'Impact_Factor', 'Impact_Factor_Source', 'Citation_Count', 'Citation_Source',
                      'DOI', 'URL', 'Abstract_En', 'Abstract_Zh', 'Abstract_Source', 'ID']

    # 重新排列列
    existing_columns = [col for col in preferred_order if col in df.columns]
    other_columns = [col for col in df.columns if col not in preferred_order]
    df = df[existing_columns + other_columns]

    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"💾 数据已保存为: {csv_filename}")

    # 生成HTML报告
    print("\n📄 生成HTML报告...")
    html_report = generate_enhanced_html_report_with_toc(final_data)
    html_filename = f"{DATE_PREFIX}_Literature_Report.html"

    with open(html_filename, 'w', encoding='utf-8') as f:
        f.write(html_report)

    print(f"✅ HTML报告已生成: {html_filename}")
    print(f"📁 文件大小: {os.path.getsize(html_filename) / 1024:.1f} KB")

    # 提供下载
    try:
        from google.colab import files
        files.download(html_filename)
        #files.download(csv_filename)  # 同时下载CSV文件
        print("⬇️ 自动下载已启动...")
    except:
        # 如果在本地环境或其他环境
        with open(html_filename, 'rb') as f:
            html_b64 = base64.b64encode(f.read()).decode()

        with open(csv_filename, 'rb') as f:
            csv_b64 = base64.b64encode(f.read()).decode()

        display(HTML(f'''
        <div style="display: flex; gap: 10px; margin: 10px 0;">
            <a href="data:text/html;base64,{html_b64}" download="{html_filename}">
                <button style="font-size:16px; padding:10px 20px; background: #4CAF50; color: white; border: none; border-radius: 5px;">
                    📄 下载HTML报告
                </button>
            </a>
            <a href="data:text/csv;base64,{csv_b64}" download="{csv_filename}">
                <button style="font-size:16px; padding:10px 20px; background: #2196F3; color: white; border: none; border-radius: 5px;">
                    📊 下载CSV数据
                </button>
            </a>
        </div>
        '''))

    return final_data

# ========== 执行主程序 ==========
if __name__ == "__main__":
    final_data = main()

📦 安装必要包...
✅ 包安装完成！
⚙️ 配置搜索参数...
🔍 关键词: ['GLP-1', 'depression']
👤 作者: ['']
🎯 搜索逻辑: (GLP-1 AND depression)
📅 时间范围: 2010-2025
📊 搜索模式: CUSTOM
🔎 作者搜索: 启用
📁 日期前缀: 20251113
📊 每源最大结果数: 5
✅ 搜索配置完成！
🎯 开始执行完整搜索流程...

📚 阶段1: 多源搜索
🚀 开始多源文献收集
🎯 搜索逻辑: (GLP-1 AND depression)
📅 时间范围: 2010-2025

🔍 正在搜索 PubMed...
🔍 Searching PubMed...
   查询: (GLP-1 AND depression) AND (2010:2025[Date - Publication])
   ✅ 找到 5 篇文章
📥 从 PubMed 添加了 5 个结果

🔍 正在搜索 CrossRef...
🔹 Searching CrossRef...
   查询: (GLP-1 AND depression)
   ✅ 找到 5 个结果
📥 从 CrossRef 添加了 5 个结果

🔍 正在搜索 Semantic Scholar...
🔹 Searching Semantic Scholar...
   查询: (GLP-1 AND depression)
   ✅ 找到 5 个结果
📥 从 Semantic Scholar 添加了 5 个结果

🎉 多源搜索完成! 总计找到 15 篇文献

🔄 阶段2: 去重

🧩 对 15 个项目进行高级去重...
📊 去重后剩余: 15 个项目
✅ 去重完成: 15 个唯一项目

🌐 阶段3: 摘要补全和翻译


📝 处理摘要: 100%|██████████| 15/15 [00:09<00:00,  1.51it/s]


📊 摘要获取统计: 改进 4, 未变 11

🔧 阶段4: 增强文献元数据

🔧 增强文献元数据...


📊 处理元数据:  60%|██████    | 9/15 [00:01<00:01,  5.97it/s]

⚠️ 增强元数据失败: name 'random' is not defined


📊 处理元数据: 100%|██████████| 15/15 [00:01<00:00,  8.25it/s]

⚠️ 增强元数据失败: name 'random' is not defined
📊 元数据增强统计:
   总文章数: 15
   有影响因子: 14 (93.3%)
   有引用次数: 6 (40.0%)
   影响因子来源: {'Original': 13, 'Unknown': 2}
   引用次数来源: {'Estimated': 7, 'Original': 6, 'Unknown': 2}

📊 最终统计:
  总唯一文章数: 15
  有DOI的文章: 14
  有摘要的文章: 15
  有影响因子的文章: 14
  有引用次数的文章: 6
  影响因子来源分布: {'Original': 13, 'Unknown': 2}
  引用次数来源分布: {'Estimated': 7, 'Original': 6, 'Unknown': 2}
💾 数据已保存为: 20251113_literature_results.csv

📄 生成HTML报告...
📄 生成带目录导航的增强HTML报告...
✅ HTML报告已生成: 20251113_Literature_Report.html
📁 文件大小: 81.9 KB


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

⬇️ 自动下载已启动...


# 新段落